In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import nltk
from nltk.stem import PorterStemmer
from sklearn.metrics.pairwise import cosine_similarity
import pickle

In [ ]:
destinasi = pd.read_csv('/content/datatourism-Fixed.csv')
# rating = pd.read_csv('/content/tourism_rating.csv')
# # user = pd.read_csv('/content/user.csv')
pd.set_option('display.max_columns', None)
destinasi
# print(destinasi.head)

,Place_Id,Place_Name,Description,Category,City,Price,Rating,Place_Ratings,Rekomendasi_Cuaca_1,Rekomendasi_Cuaca_2,Rekomendasi_Cuaca_3,Rekomendasi_Cuaca_4,Rekomendasi_Cuaca_5
0,1,Monumen Nasional,Monumen Nasional atau yang populer disingkat d...,Budaya,Jakarta,20000,4.6,3.722222,Hujan Ringan,Berawan,Cerah,Hujan Sedang,Cerah Berawan
1,2,Kota Tua,"Kota tua di Jakarta, yang juga bernama Kota Tu...",Budaya,Jakarta,0,4.6,2.840000,Cerah,Cerah Berawan,Berawan,Hujan Ringan,Hujan Lebat
2,3,Dunia Fantasi,Dunia Fantasi atau disebut juga Dufan adalah t...,Taman Hiburan,Jakarta,270000,4.6,2.526316,Cerah,Cerah Berawan,Berawan,Hujan Ringan,Hujan Lebat
3,4,Taman Mini Indonesia Indah (TMII),Taman Mini Indonesia Indah merupakan suatu kaw...,Taman Hiburan,Jakarta,10000,4.5,2.857143,Cerah,Cerah Berawan,Berawan,Hujan Ringan,Berawan Tebal
4,5,Atlantis Water Adventure,Atlantis Water Adventure atau dikenal dengan A...,Taman Hiburan,Jakarta,94000,4.5,3.520000,Cerah,Cerah Berawan,Berawan,Hujan Ringan,Hujan Lebat
...,...,...,...,...,...,...,...,...,...,...,...,...,...
432,433,Museum Mpu Tantular,Museum Negeri Mpu Tantular adalah sebuah museu...,Budaya,Surabaya,2000,4.4,3.304348,Hujan Ringan,Cerah,Hujan Sedang,Cerah Berawan,Berawan
433,434,Taman Bungkul,Taman Bungkul adalah taman wisata kota yang te...,Taman Hiburan,Surabaya,0,4.6,2.400000,Cerah,Cerah Berawan,Berawan,Hujan Ringan,Berawan Tebal
434,435,Taman Air Mancur Menari Kenjeran,Air mancur menari atau dancing fountain juga a...,Taman Hiburan,Surabaya,0,4.4,3.000000,Cerah,Cerah Berawan,Berawan,Hujan Ringan,Berawan Tebal
435,436,Taman Flora Bratang Surabaya,Taman Flora adalah salah satu taman kota di Su...,Taman Hiburan,Surabaya,0,4.6,3.090909,Cerah,Cerah Berawan,Berawan,Hujan Ringan,Berawan Tebal


In [ ]:
destinasi['is_decimal'] = destinasi['Rating'].astype(float).apply(lambda x: x % 1 != 0)

# Hitung jumlah nilai False (yang berarti bukan desimal)
jumlah_non_decimal = (destinasi['is_decimal'] == False).sum()

# Menampilkan hasil
print("Jumlah nilai yang bukan desimal:", jumlah_non_decimal)

Jumlah nilai yang bukan desimal: 22


In [ ]:
destinasi['Rating'] = destinasi['Rating'].astype(float).round(1)
destinasi['Place_Ratings'] = destinasi['Place_Ratings'].astype(float).round(2)

print(destinasi)

     Place_Id                                 Place_Name  \
0           1                           Monumen Nasional   
1           2                                   Kota Tua   
2           3                              Dunia Fantasi   
3           4          Taman Mini Indonesia Indah (TMII)   
4           5                   Atlantis Water Adventure   
..        ...                                        ...   
432       433                        Museum Mpu Tantular   
433       434                              Taman Bungkul   
434       435           Taman Air Mancur Menari Kenjeran   
435       436               Taman Flora Bratang Surabaya   
436       437  Gereja Perawan Maria Tak Berdosa Surabaya   

                                           Description       Category  \
0    Monumen Nasional atau yang populer disingkat d...         Budaya   
1    Kota tua di Jakarta, yang juga bernama Kota Tu...         Budaya   
2    Dunia Fantasi atau disebut juga Dufan adalah t...  Tama

In [ ]:
# prompt: Using dataframe destinasi: drop column Rekomendasi Cuaca 5 and Rekomendasi Cuaca 4

# Drop the specified columns
destinasi = destinasi.drop(columns=['Price'])
# Display the updated dataframe (optional)
print(destinasi.head())


In [ ]:
destinasi = destinasi.drop(columns='is_decimal')

In [ ]:
print(destinasi.head())

   Place_Id                         Place_Name  \
0         1                   Monumen Nasional   
1         2                           Kota Tua   
2         3                      Dunia Fantasi   
3         4  Taman Mini Indonesia Indah (TMII)   
4         5           Atlantis Water Adventure   

                                         Description       Category     City  \
0  Monumen Nasional atau yang populer disingkat d...         Budaya  Jakarta   
1  Kota tua di Jakarta, yang juga bernama Kota Tu...         Budaya  Jakarta   
2  Dunia Fantasi atau disebut juga Dufan adalah t...  Taman Hiburan  Jakarta   
3  Taman Mini Indonesia Indah merupakan suatu kaw...  Taman Hiburan  Jakarta   
4  Atlantis Water Adventure atau dikenal dengan A...  Taman Hiburan  Jakarta   

   Rating  Place_Ratings Rekomendasi_Cuaca_1 Rekomendasi_Cuaca_2  \
0     4.6           3.72        Hujan Ringan             Berawan   
1     4.6           2.84               Cerah       Cerah Berawan   
2     4.6     

In [ ]:
target_columns = ['Rekomendasi_Cuaca_1', 'Rekomendasi_Cuaca_2', 'Rekomendasi_Cuaca_3', 'Rekomendasi_Cuaca_4', 'Rekomendasi_Cuaca_5']

# Mengganti 'kabut' dan 'hujan lebat' dengan NaN atau nilai kosong
destinasi[target_columns] = destinasi[target_columns].replace(['Kabut', 'Hujan Lebat'], '', regex=False)
destinasi[target_columns] = destinasi[target_columns].replace(['Cerah Berawan'], 'Cerah_Berawan', regex=False)
destinasi[target_columns] = destinasi[target_columns].replace(['Berawan Tebal'], 'Berawan_Tebal', regex=False)
destinasi[target_columns] = destinasi[target_columns].replace(['Hujan Ringan'], 'Hujan_Ringan', regex=False)
destinasi[target_columns] = destinasi[target_columns].replace(['Hujan Sedang'], 'Hujan_Sedang', regex=False)

print(destinasi)

     Place_Id                                 Place_Name  \
0           1                           Monumen Nasional   
1           2                                   Kota Tua   
2           3                              Dunia Fantasi   
3           4          Taman Mini Indonesia Indah (TMII)   
4           5                   Atlantis Water Adventure   
..        ...                                        ...   
432       433                        Museum Mpu Tantular   
433       434                              Taman Bungkul   
434       435           Taman Air Mancur Menari Kenjeran   
435       436               Taman Flora Bratang Surabaya   
436       437  Gereja Perawan Maria Tak Berdosa Surabaya   

                                           Description       Category  \
0    Monumen Nasional atau yang populer disingkat d...         Budaya   
1    Kota tua di Jakarta, yang juga bernama Kota Tu...         Budaya   
2    Dunia Fantasi atau disebut juga Dufan adalah t...  Tama

In [ ]:
destinasi.head()

In [ ]:
target_columns = ['Rekomendasi_Cuaca_1', 'Rekomendasi_Cuaca_2', 'Rekomendasi_Cuaca_3', 'Rekomendasi_Cuaca_4', 'Rekomendasi_Cuaca_5']

# Menggabungkan nilai dari lima kolom, dipisahkan oleh koma, mengabaikan nilai kosong
destinasi['rekomendasi'] = destinasi[target_columns].apply(
    lambda row: ', '.join(filter(lambda x: x.strip() != '', row)), axis=1
)

destinasi.drop(columns=target_columns, inplace=True)

print(destinasi)

     Place_Id                                 Place_Name  \
0           1                           Monumen Nasional   
1           2                                   Kota Tua   
2           3                              Dunia Fantasi   
3           4          Taman Mini Indonesia Indah (TMII)   
4           5                   Atlantis Water Adventure   
..        ...                                        ...   
432       433                        Museum Mpu Tantular   
433       434                              Taman Bungkul   
434       435           Taman Air Mancur Menari Kenjeran   
435       436               Taman Flora Bratang Surabaya   
436       437  Gereja Perawan Maria Tak Berdosa Surabaya   

                                           Description       Category  \
0    Monumen Nasional atau yang populer disingkat d...         Budaya   
1    Kota tua di Jakarta, yang juga bernama Kota Tu...         Budaya   
2    Dunia Fantasi atau disebut juga Dufan adalah t...  Tama

In [ ]:
print(destinasi.isnull().sum())

Place_Id         0
Place_Name       0
Description      0
Category         0
City             0
Rating           0
Place_Ratings    0
rekomendasi      0
dtype: int64


In [ ]:
preparation = destinasi
preparation.sort_values('Place_Id')

In [ ]:
all_tourism_name = destinasi
all_tourism_name

,Place_Id,Place_Name,Description,Category,City,Rating,Place_Ratings,rekomendasi
0,1,Monumen Nasional,Monumen Nasional atau yang populer disingkat d...,Budaya,Jakarta,4.6,3.72,"Hujan_Ringan, Berawan, Cerah, Hujan_Sedang, Ce..."
1,2,Kota Tua,"Kota tua di Jakarta, yang juga bernama Kota Tu...",Budaya,Jakarta,4.6,2.84,"Cerah, Cerah_Berawan, Berawan, Hujan_Ringan"
2,3,Dunia Fantasi,Dunia Fantasi atau disebut juga Dufan adalah t...,Taman Hiburan,Jakarta,4.6,2.53,"Cerah, Cerah_Berawan, Berawan, Hujan_Ringan"
3,4,Taman Mini Indonesia Indah (TMII),Taman Mini Indonesia Indah merupakan suatu kaw...,Taman Hiburan,Jakarta,4.5,2.86,"Cerah, Cerah_Berawan, Berawan, Hujan_Ringan, B..."
4,5,Atlantis Water Adventure,Atlantis Water Adventure atau dikenal dengan A...,Taman Hiburan,Jakarta,4.5,3.52,"Cerah, Cerah_Berawan, Berawan, Hujan_Ringan"
...,...,...,...,...,...,...,...,...
432,433,Museum Mpu Tantular,Museum Negeri Mpu Tantular adalah sebuah museu...,Budaya,Surabaya,4.4,3.30,"Hujan_Ringan, Cerah, Hujan_Sedang, Cerah_Beraw..."
433,434,Taman Bungkul,Taman Bungkul adalah taman wisata kota yang te...,Taman Hiburan,Surabaya,4.6,2.40,"Cerah, Cerah_Berawan, Berawan, Hujan_Ringan, B..."
434,435,Taman Air Mancur Menari Kenjeran,Air mancur menari atau dancing fountain juga a...,Taman Hiburan,Surabaya,4.4,3.00,"Cerah, Cerah_Berawan, Berawan, Hujan_Ringan, B..."
435,436,Taman Flora Bratang Surabaya,Taman Flora adalah salah satu taman kota di Su...,Taman Hiburan,Surabaya,4.6,3.09,"Cerah, Cerah_Berawan, Berawan, Hujan_Ringan, B..."


In [ ]:
all_tourism_name.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 437 entries, 0 to 436
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Place_Id       437 non-null    int64  
 1   Place_Name     437 non-null    object 
 2   Description    437 non-null    object 
 3   Category       437 non-null    object 
 4   City           437 non-null    object 
 5   Rating         437 non-null    float64
 6   Place_Ratings  437 non-null    float64
 7   rekomendasi    437 non-null    object 
dtypes: float64(2), int64(1), object(5)
memory usage: 27.4+ KB


In [ ]:
preparation = all_tourism_name
preparation.sort_values('Place_Id')

,Place_Id,Place_Name,Description,Category,City,Rating,Place_Ratings,rekomendasi
0,1,Monumen Nasional,Monumen Nasional atau yang populer disingkat d...,Budaya,Jakarta,4.6,3.72,"Hujan Ringan, Berawan, Cerah, Hujan Sedang, Ce..."
1,2,Kota Tua,"Kota tua di Jakarta, yang juga bernama Kota Tu...",Budaya,Jakarta,4.6,2.84,"Cerah, Cerah Berawan, Berawan, Hujan Ringan"
2,3,Dunia Fantasi,Dunia Fantasi atau disebut juga Dufan adalah t...,Taman Hiburan,Jakarta,4.6,2.53,"Cerah, Cerah Berawan, Berawan, Hujan Ringan"
3,4,Taman Mini Indonesia Indah (TMII),Taman Mini Indonesia Indah merupakan suatu kaw...,Taman Hiburan,Jakarta,4.5,2.86,"Cerah, Cerah Berawan, Berawan, Hujan Ringan, B..."
4,5,Atlantis Water Adventure,Atlantis Water Adventure atau dikenal dengan A...,Taman Hiburan,Jakarta,4.5,3.52,"Cerah, Cerah Berawan, Berawan, Hujan Ringan"
...,...,...,...,...,...,...,...,...
432,433,Museum Mpu Tantular,Museum Negeri Mpu Tantular adalah sebuah museu...,Budaya,Surabaya,4.4,3.30,"Hujan Ringan, Cerah, Hujan Sedang, Cerah Beraw..."
433,434,Taman Bungkul,Taman Bungkul adalah taman wisata kota yang te...,Taman Hiburan,Surabaya,4.6,2.40,"Cerah, Cerah Berawan, Berawan, Hujan Ringan, B..."
434,435,Taman Air Mancur Menari Kenjeran,Air mancur menari atau dancing fountain juga a...,Taman Hiburan,Surabaya,4.4,3.00,"Cerah, Cerah Berawan, Berawan, Hujan Ringan, B..."
435,436,Taman Flora Bratang Surabaya,Taman Flora adalah salah satu taman kota di Su...,Taman Hiburan,Surabaya,4.6,3.09,"Cerah, Cerah Berawan, Berawan, Hujan Ringan, B..."


In [ ]:
place_id = preparation['Place_Id'].tolist()
place_category = preparation['Category'].tolist()
place_name = preparation['Place_Name'].tolist()
place_description = preparation['Description'].tolist()
place_city = preparation['City'].tolist()
place_rekomendasi = preparation['rekomendasi'].tolist()

print(len(place_id))
print(len(place_category))
print(len(place_name))
print(len(place_description))
print(len(place_city))
print(len(place_rekomendasi))

437
437
437
437
437
437


In [ ]:
tourism_recommend = pd.DataFrame({
    'id' : place_id,
    'place_category' : place_category,
    'place_name' : place_name,
    'place_description' : place_description,
    'place_city' : place_city,
    'place_rekomendasi' : place_rekomendasi
})

tourism_recommend

In [ ]:
data = tourism_recommend
data.head()

NameError: name 'tourism_recommend' is not defined

In [ ]:
tfidf = TfidfVectorizer()
tfdif_matrix = tfidf.fit_transform(data['place_name'])
tfdif_matrix.shape

In [ ]:
tfdif_matrix.todense()

In [ ]:
pd.DataFrame(
    tfdif_matrix.todense(),
    columns = tfidf.get_feature_names_out(),
    index = data.place_category
).sample(22, axis = 1).sample(10, axis = 0)

In [ ]:
cosine_sim = cosine_similarity(tfdif_matrix)
cosine_sim

In [ ]:
cosine_sim_df = pd.DataFrame(cosine_sim, index = data['place_name'], columns = data['place_name'])
print('Shape : ', cosine_sim_df.shape)

cosine_sim_df.sample(5, axis = 1).sample(10, axis = 0)

In [ ]:
def tourism_recommendations(nama_tempat, similarity_data = cosine_sim_df, items = data[['place_name', 'place_category','place_city','place_rekomendasi']], k = 5) :
  index = similarity_data.loc[:, nama_tempat].to_numpy().argpartition(
      range(-1, -k, -1)
  )

  closest = similarity_data.columns[index[-1:-(k+2):-1]]
  closest = closest.drop(nama_tempat, errors = 'ignore')

  return pd.DataFrame(closest).merge(items).head(k)

In [ ]:
data[data.place_name.eq('Masjid Nasional Al-Akbar')]

,id,place_category,place_name,place_description,place_city,place_rekomendasi
411,412,Tempat Ibadah,Masjid Nasional Al-Akbar,Masjid Nasional Al Akbar (atau biasa disebut M...,Surabaya,"Hujan Ringan, Hujan Sedang, Cerah, Berawan, Hu..."


In [ ]:
tourism_recommendations('Masjid Nasional Al-Akbar')

,place_name,place_category,place_city,place_rekomendasi
0,Masjid Al-Imtizaj,Tempat Ibadah,Bandung,"Hujan Ringan, Hujan Sedang, Cerah, Berawan, Hu..."
1,Museum Nasional,Budaya,Jakarta,"Hujan Ringan, Cerah, Hujan Sedang, Cerah Beraw..."
2,Monumen Nasional,Budaya,Jakarta,"Hujan Ringan, Berawan, Cerah, Hujan Sedang, Ce..."
3,Pantai Ngobaran,Bahari,Yogyakarta,"Cerah, Cerah Berawan, Berawan, Berawan Tebal, ..."
4,Pantai Ngrawe (Mesra),Bahari,Yogyakarta,"Cerah, Cerah Berawan, Berawan, Berawan Tebal, ..."


In [ ]:
all_tourism_name['combined_text'] = all_tourism_name['Description'] + ' ' + all_tourism_name['Category'] + ' ' + all_tourism_name['City'] +' '+ all_tourism_name['rekomendasi']
#Fill null with empty string to avoid issues
all_tourism_name['combined_text'] = all_tourism_name['combined_text'].fillna('')

In [ ]:
#Instantiate TF-IDF Vectorizer
vectorizer = TfidfVectorizer(stop_words='english', max_df=0.95, min_df=2, ngram_range=(1, 1))

vectorizer.fit(all_tourism_name['combined_text'])
vectorizer.get_feature_names_out()

array(['00', '000', '01', ..., 'ꦥꦫ', 'ꦥꦱ', 'ꦩꦫ'], dtype=object)

In [ ]:
tfidf_matrix = vectorizer.fit_transform(all_tourism_name['combined_text'])
tfidf_matrix.shape

(437, 2707)

In [ ]:
tfidf_matrix.todense()

matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
# Compute the cosine similarity matrix based on the tfidf_matrix
cosine_sim = cosine_similarity(tfidf_matrix)

# Print the shape of the cosine similarity matrix to verify
print(cosine_sim.shape)
print()

cosine_sim

(437, 437)



array([[1.        , 0.07544402, 0.16179105, ..., 0.06624022, 0.04846971,
        0.01431997],
       [0.07544402, 1.        , 0.09018642, ..., 0.03081298, 0.02245853,
        0.07597372],
       [0.16179105, 0.09018642, 1.        , ..., 0.07536258, 0.10564481,
        0.01946812],
       ...,
       [0.06624022, 0.03081298, 0.07536258, ..., 1.        , 0.12023572,
        0.09880175],
       [0.04846971, 0.02245853, 0.10564481, ..., 0.12023572, 1.        ,
        0.19377918],
       [0.01431997, 0.07597372, 0.01946812, ..., 0.09880175, 0.19377918,
        1.        ]])

In [ ]:
def recommend(id_place, rekomendasi=None, top_n=5):
    # Mengambil indeks tempat jika Place_Id diberikan
    if id_place:
        idx = all_tourism_name.index[all_tourism_name['Place_Id'] == id_place][0]
        sim_scores = list(enumerate(cosine_sim[idx]))
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        sim_scores = sim_scores[1:]  # Hindari tempat itu sendiri
        city_of_place = all_tourism_name.iloc[idx]['City']  # Kota dari tempat wisata input
    else:
        # Jika hanya rekomendasi diberikan, gunakan seluruh dataset
        sim_scores = list(enumerate(cosine_sim[0]))

    # Filter berdasarkan rekomendasi jika rekomendasi diberikan
    if rekomendasi:
        rekomendasi = rekomendasi.lower().replace(" ", "")
        place_indices = [
            i[0] for i in sim_scores
            if rekomendasi in all_tourism_name.iloc[i[0]]['rekomendasi'].lower()
        ]
    else:
        place_indices = [i[0] for i in sim_scores]

    place_indices = [
        i for i in place_indices
        if all_tourism_name.iloc[i]['City'].lower() == city_of_place.lower()
    ]


    place_indices = place_indices[:top_n]

    return all_tourism_name['Place_Name'].iloc[place_indices]

In [ ]:
print(recommend(id_place=15, rekomendasi='Hujan_Sedang'))

38    Museum Macan (Modern and Contemporary Art in N...
41                          Museum Seni Rupa dan Kramik
76                                    Taman Hutan Tebet
16                                    Museum Fatahillah
59                                       Museum Tekstil
Name: Place_Name, dtype: object


In [ ]:
pickle.dump(cosine_sim, open('cosine_sim.pkl', 'wb'))
pickle.dump(all_tourism_name, open('place.pkl', 'wb'))

In [ ]:
cosine_sim = pickle.load(open('cosine_sim.pkl', 'rb'))

lenn = len(cosine_sim)

cosine_sim1 = cosine_sim[:lenn//3]
cosine_sim2 = cosine_sim[lenn//3:2*lenn//3]
cosine_sim3 = cosine_sim[2*lenn//3:]

pickle.dump(cosine_sim1, open('cosine_sim1.pkl', 'wb'))
pickle.dump(cosine_sim2, open('cosine_sim2.pkl', 'wb'))
pickle.dump(cosine_sim3, open('cosine_sim3.pkl', 'wb'))

In [ ]:
mkdir model

In [ ]:
# prompt: zip folder

!zip -r /content/model.zip /content/model


  adding: content/model/ (stored 0%)
  adding: content/model/place.pkl (deflated 68%)
  adding: content/model/cosine_sim1.pkl (deflated 7%)
  adding: content/model/cosine_sim2.pkl (deflated 7%)
  adding: content/model/cosine_sim.pkl (deflated 7%)
  adding: content/model/cosine_sim3.pkl (deflated 7%)
